# Diffusion tensor model

The main limitation of the previously mentioned diffusion model is that it does not perform well in anisotropic areas of the brain since it does not take gradient direction into account and thus cannot estimate the direction of diffusion. The diffusion tensor model is a direct extension of the previous model to 3D. The implicit assumption of the DT model is that the dispersion pattern is a trivariate Gaussian. Therefore DT is proportional to the covariance matrix of the distribution. The positive-definitivness of DT ensures that the diffusivity will always be positive (negative diffusivity is meaningless).

Diffusion in the DT model is defined as a quadratic function of gradient direction:

$$d(\hat{q})=\hat{q}^{T}D\hat{q}$$

where $D$ is a symmetric positive-definite $3\times3$ tensor of parameters that need to estimated:

$$D=\begin{pmatrix}
D_{xx}&  D_{xy}& D_{xz}\\ 
D_{xy}&  D_{yy}& D_{yz}\\ 
D_{xz}&  D_{yz}& D_{zz}
\end{pmatrix}$$

Hence the DT model can be defined as:

$$S(b,\hat{q})=S(0,\mathbf{0})e^{-b\hat{q}^{T}D\hat{q}}$$

Now the model can be linearized:

$$logS(b,\hat{q})=logS(0,\mathbf{0})-b\hat{q}^{T}D\hat{q}$$

Finally the terms can be expanded:

$$logS(b,\hat{q})=logS(0,\mathbf{0}) - bq^{2}_{x}D_{xx} - 2bq_{x}q_{y}D_{xy} - 2bq_{x}q_{z}D_{xz} - bq_{y}^{2}D_{yy} - 2bq_{y}q_{z}D_{yz} - bq_{z}^{2}D_{zz}$$

where $q_x$, $q_y$ and $q_z$ are the $x$, $y$ and $z$ components of $\hat{q}$

This expansion can be used to establish a system of linear equations:

$$\begin{pmatrix}
logA(b_{1},\hat{q}_{1})\\ 
\vdots \\ 
logA(b_{n},\hat{q}_{n})
\end{pmatrix} = 
\begin{pmatrix}
1 & -b_{1}q^{2}_{1x} & -2b_{1}q_{1x}q_{1y} & -2b_{1}q_{1x}q_{1z} & -b_{1}q_{1y}^{2} & -2b_{1}q_{1y}q_{1z} &-b_{1}q_{1z}^{2}\\ 
\vdots &\vdots &\vdots &\vdots &\vdots &\vdots &\vdots \\ 
1 & -b_{n}q^{2}_{nx} & -2b_{n}q_{nx}q_{ny} & -2b_{n}q_{nx}q_{nz} & -b_{n}q_{ny}^{2} & -2b_{n}q_{ny}q_{nz} &-b_{n}q_{nz}^{2}
\end{pmatrix}
\begin{pmatrix}
logS(0,\mathbf{0})\\ 
D_{xx}\\ 
D_{xy}\\ 
D_{xz}\\ 
D_{yy}\\ 
D_{yz}\\ 
D_{zz}
\end{pmatrix}
$$

where $n$ is the number of measurements. This is again ordinary least squares problem which can be solved with Moore-Penrose pseudoinverse.

The elements of $D$ on the main diagonal show diffusivity in the $x$, $y$ and $z$ directions.

The figure below demonstrates that the DT model achieves significantly lower error in the anisotropic regions than the simple diffusion model:

![SIMPLE_DIFFUSION_MODEL_PERFORMANCE](./figures/diffusion_tensor_model_performance.png)